# Glider Toolbox Demo Notebook

### Using a Pre-existing config file for a complete pipeline run

Note: Currently these imports are local references. In the future the toolbox will be an installable package through pip.

In [1]:
# add toolbox to src (THIS WILL NOT BE REQUIRED ONCE DEPLOYED)
import sys
from pathlib import Path

src_path = Path("../../src").resolve()

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

### Pipeline Overview

In [5]:
from toolbox.pipeline import Pipeline
pipeline = Pipeline(
    "../configs/example_config_nelson.yaml")
pipeline.run()

Step 'Load OG1' added successfully!
Step 'Derive CTD' added successfully!
Step 'Find Profiles' added successfully!
Step 'Data Export' added successfully!
Executing: Load OG1
[Load OG1] Params: {'file_path': '/home/aaron-mau/Code/NOC-toolbox/mau/noc_toolbox/data/sea045_20230530T0832_delayed.nc', 'add_meta': False, 'add_elapsed_time': False, 'add_dev_cols': False}
[Load OG1] Loading /home/aaron-mau/Code/NOC-toolbox/mau/noc_toolbox/data/sea045_20230530T0832_delayed.nc OG1
[Load OG1] Diagnostics: False
[Derive CTD] Data found in context.
Executing: Derive CTD
[Derive CTD] Processing CTD...
[Derive CTD] Deriving DEPTH...
{'DEPTH_QC': ['PRES_QC', 'LATITUDE_QC']}
I made it inside!
DEPTH_QC
['PRES_QC', 'LATITUDE_QC']
Got to before the combinatrix. qc_parents: ['PRES_QC', 'LATITUDE_QC']
PRES_QC
LATITUDE_QC


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

#### Building the pipeline one step at a time

In [ ]:
# pipeline2 = Pipeline()

# class myCustomStep:
#     def __init__(self, param1, param2):
#         self.param1 = param1
#         self.param2 = param2

#     def run(self, data):
#         # Custom processing logic here
#         print(f"Running custom step with {self.param1} and {self.param2}")
#         return data  # Return processed data

# # add load step
# pipeline2.add_step(
#     "Load OG1",
#     parameters={
#         "file_path": "../../examples/data/OG1/Churchill_647_R.nc",  # Path to the input NetCDF file
#         "add_meta": False, # Add metadata to the data
#         "add_depth": True, # Add depth information to the data
#         "add_elapsed_time": False,
#         "lat_label": "DEPLOYMENT_LATITUDE"},
#     diagnostics=False,
#     run_immediately=True )# can run immediately upon creation

# # add export step
# pipeline2.add_step(
#     "Data Export",
#     parameters={
#         "export_format": "netcdf",  # Define the export format (e.g., OG1, CSV, etc.)
#         "output_path": "../../examples/data/OG1/exported_Churchill_647_R.nc"},
#     diagnostics=False,
#     run_immediately=False )# or run at a future time
# # run last step
# pipeline2.run_last_step()

In [ ]:
# conf = pipeline2.export_config("../../examples/data/OG1/Churchill_647_R_CONFIG.yaml")  # Export the pipeline configuration to a YAML file
# conf

# Running Multiple Pipelines

```python

In [ ]:
### Running multiple pipelines using the PipelineManager class
from toolbox.pipeline import PipelineManager

mngr = PipelineManager()
mngr.load_mission_control(
    "../configs/MissionControl.yaml"
)  # Load the mission control configuration
# This is currently set up to import data, run profiles, and export data for Doombar and Churchill sources.
mngr.run_all()  # Run all pipelines defined in the mission control configuration
mngr.summarise_all_profiles()

In [ ]:
mngr.preview_alignment("Cabot")  # Align data to the target source (e.g., Doombar)

In [ ]:
mngr.fit_and_save_to_target("Cabot")  # Fit data to the target source (e.g., Doombar)

In [ ]:
mngr.validate_with_device("Cabot")

In [ ]:
mngr.fit_to_device("Cabot")

In [ ]:
mngr.save_outputs("Cabot")